<h1>Installs</h1>
- This file prepares the data such that we can upload it to opensearch (So that we can have vecotrs and metadata all in one DB). We use the same e5 model as in strategy_1 but chunk the data with the SentenceTransformersTokenTextSplitter method. 

In [1]:
! pip install pinecone-client
! pip install faunadb
! pip install ndg-httpsclient
! pip install pyopenssl
! pip install pyasn1
! pip install -U sentence-transformers
! pip install transformers


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: C:\Users\Admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: C:\Users\Admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: C:\Users\Admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: C:\Users\Admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: C:\Users\Admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


  Obtaining dependency information for sentence-transformers from https://files.pythonhosted.org/packages/06/97/57afa3d05801b6b9305f96a7ce5995e12c1d2ba25ce66747de107816b0b5/sentence_transformers-2.3.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/132.8 kB ? eta -:--:--
   --- ------------------------------------ 10.2/132.8 kB ? eta -:--:--
   ------------------ -------------------- 61.4/132.8 kB 825.8 kB/s eta 0:00:01
   ------------------ -------------------- 61.4/132.8 kB 825.8 kB/s eta 0:00:01
   --------------------- ----------------- 71.7/132.8 kB 491.5 kB/s eta 0:00:01
   -------------------------------------- 132.8/132.8 kB 712.7 kB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 2.2.2
    Uninstalling sentence-transformers-2.2.2:
      Successfully uninstalled sentence-transformers-2.2.2



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: C:\Users\Admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: C:\Users\Admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



<h1>Extract Data from .csv</h1>
*   AB = Column that corresponds to the text of the abstract


In [4]:
import csv
from itertools import islice
import uuid

# Load data, store abstract text
batch_input = []
with open('../../../articles.csv', encoding = "utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        batch_input.append([row["AB"],row["PMID"],row["TI"],row["PB"],row["FAU"],row["FED"],row["DP"],row["OTO"], row["OT"],row["OWN"], row["LR"], row["JT"], row["MH"],row["ISBN"]])

In [5]:
print(len(batch_input))

57560



<h1>Embedd Data from .csv</h1>
1. DIvide each abstract into chunks such that each chunks text will be tokenized to max 512 Tokens


In [2]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer
from langchain.text_splitter import SentenceTransformersTokenTextSplitter
import re
import tqdm

#embed data
# model = SentenceTransformer('pritamdeka/S-PubMedBert-MS-MARCO')
# tokenizer = AutoTokenizer.from_pretrained('pritamdeka/S-PubMedBert-MS-MARCO')
# max_token_size = 350
# print(model)
# print(tokenizer)

model_name = 'intfloat/e5-base-v2'
model = SentenceTransformer(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
max_token_size = 512
print(model)
print(tokenizer)


C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
modules.json: 100%|██████████| 387/387 [00:00<?, ?B/s] 
C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--intfloat--e5-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For mo

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
  (2): Normalize()
)
BertTokenizerFast(name_or_path='intfloat/e5-base-v2', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstr

In [6]:
splitter = SentenceTransformersTokenTextSplitter(
    model_name=model_name,  
    chunk_overlap=20,  
)
print(splitter.maximum_tokens_per_chunk)

512


In [7]:
chunked_data = []

#Chunk abstract text, prepare for emebdding
for abstract_text in batch_input:
    chunked_data.append(splitter.split_text(text=abstract_text[0]))

In [8]:
print(len(chunked_data))

57560


In [9]:
chunks = []

#flatten data and append metadata to each chunk -> list of [chunk, PMID, TI, PB, FAU, FED,DP,OTO,ISBN]
for id, abstract in enumerate(chunked_data):
    for chunk in abstract:
        chunks.append([chunk, batch_input[id][1],batch_input[id][2],batch_input[id][3],batch_input[id][4],batch_input[id][5],batch_input[id][6],batch_input[id][7],batch_input[id][8],batch_input[id][9],batch_input[id][10],batch_input[id][11],batch_input[id][12]])

In [10]:
print(len(chunks))

62333


In [11]:
from tqdm import tqdm

In [ ]:
#add embedding vector of chunk to the list
for chunk in tqdm(chunks):
    chunk.insert(0, model.encode(chunk[0]).tolist())

In [ ]:
import pickle

#save List in File. This file will be later accesssed and the data will be uploaded to open search

path = "./data/data.txt"
with open(path, 'wb') as file:
    pickle.dump(chunks, file)